#IMPORTACION DE HERRAMIENTAS A USAR

In [294]:
import pandas as pd #importamos pandas para los dataframes
import numpy as np # pa los calculos numéricos
import matplotlib.pyplot as plt #pa los graficos

#LECTURA DE ARCHIVO A USAR

In [295]:
pd.read_csv("encuestas_1000_casos.csv") #leemos el csv
df=pd.read_csv("encuestas_1000_casos.csv") #lo designamos a una dataframe

#CHEQUEO NULOS Y DATA GRAL

In [296]:
print(df.isnull().sum()) #vemos cuantos nulos hay

Fecha                                   0
Encuesta                                0
Estrato                                 0
Sexo                                    0
Edad                                   48
Nivel Educativo                        21
Cantidad de Integrantes en el Hogar     0
Imagen del Candidato                    8
Voto                                   30
Voto Anterior                           0
dtype: int64


In [297]:
print(df.columns.tolist()) #vemos las columnas como una lista
print(df.info()) #info general del df

['Fecha', 'Encuesta', 'Estrato', 'Sexo', 'Edad', 'Nivel Educativo', 'Cantidad de Integrantes en el Hogar', 'Imagen del Candidato', 'Voto', 'Voto Anterior']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Fecha                                1000 non-null   object 
 1   Encuesta                             1000 non-null   int64  
 2   Estrato                              1000 non-null   object 
 3   Sexo                                 1000 non-null   object 
 4   Edad                                 952 non-null    float64
 5   Nivel Educativo                      979 non-null    object 
 6   Cantidad de Integrantes en el Hogar  1000 non-null   int64  
 7   Imagen del Candidato                 992 non-null    float64
 8   Voto                                 970 non-null    object 
 9   Voto An

#NORMALIZACION DE DATOS

In [298]:
[col for col in df.columns if "fecha" in col.lower()] # buscamos columna fecha para ver nombre exacto
#como la fecha va a ser lo que indexemos la unificamos antes que nada
df.columns = df.columns.str.lower().str.replace(" ", "_") # normalizamos nombres columnas
df = df.drop_duplicates() # eliminamos filas duplicadas
print(df.columns.tolist()) # printeamos los cambios de nombres de las columnas


['fecha', 'encuesta', 'estrato', 'sexo', 'edad', 'nivel_educativo', 'cantidad_de_integrantes_en_el_hogar', 'imagen_del_candidato', 'voto', 'voto_anterior']


In [299]:
print(df.isnull().sum())  # vemos que no hay nulos
print(df.info()) # mostramos informacion del dataframe luego de la limpieza
print(df.describe())# mostramos estadisticas descriptivas del dataframe


fecha                                   0
encuesta                                0
estrato                                 0
sexo                                    0
edad                                   47
nivel_educativo                        21
cantidad_de_integrantes_en_el_hogar     0
imagen_del_candidato                    8
voto                                   30
voto_anterior                           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 841 entries, 0 to 997
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   fecha                                841 non-null    object 
 1   encuesta                             841 non-null    int64  
 2   estrato                              841 non-null    object 
 3   sexo                                 841 non-null    object 
 4   edad                                 794 non-null    float64
 5   

In [300]:
df["edad"] = df["edad"].astype("Int64") # convertimos la columna edad a entero
print(df["edad"].dtype) # verificamos el cambio de tipo de dato en la columna

Int64


In [301]:
#como la fecha dice objets significa que la columna es string y no sirve para rolling por tiempo entonces:
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce') # convertimos la columna 'fecha' a tipo datetime
df = df.dropna(subset=['fecha']) # eliminamos filas donde 'fecha' no se pudo convertir  a datetime
# podemos llenar con la media o la moda en vez de eliminar nulos, pero en este caso no hay nulos en fecha, pero si puede ser en otras columnas para el analisis posterior
print(df.columns.tolist())
print(df["fecha"].dtype) #confirmamos que la fecha sea el formato correcto para poder aplicar rolling window

['fecha', 'encuesta', 'estrato', 'sexo', 'edad', 'nivel_educativo', 'cantidad_de_integrantes_en_el_hogar', 'imagen_del_candidato', 'voto', 'voto_anterior']
datetime64[ns]


In [302]:
pd.read_csv("censo_2022_v_final.csv") #leemos el csv del censo
df_censo=pd.read_csv("censo_2022_v_final.csv") #lo designamos a una dataframe

In [303]:
#NORMALIZAMOS COLUMNAS PARA QUE COINCIDAN
# ============================================================

# normalizamos censo
df_censo.columns = df_censo.columns.str.lower().str.strip()
df_censo['estrato'] = df_censo['estrato'].str.lower().str.strip()
df_censo['sexo'] = df_censo['sexo'].str.lower().str.strip()

# normalizamos encuesta
df.columns = df.columns.str.lower().str.strip()
df['estrato'] = df['estrato'].str.lower().str.strip()
df['sexo'] = df['sexo'].str.lower().str.strip()

# ============================================================
# 3) CALCULAMOS EL PESO DEL CENSO
# ============================================================

# población total del país según el archivo
poblacion_total = df_censo['poblacion'].sum()

# peso = proporción de población que representa ese grupo (estrato+sexo+edad)
df_censo['peso'] = df_censo['poblacion'] / poblacion_total

# esta es la tabla maestra que vamos a mergear
tabla_pesos = df_censo[['estrato','sexo','edad','peso']]

# ============================================================
# 4) MERGE PARA ASIGNAR PESO A CADA ENCUESTA
# ============================================================

df = df.merge(
    tabla_pesos,
    on=['estrato','sexo','edad'],   # claves comunes
    how='left'
)

# ============================================================
# 5) VERIFICAMOS
# ============================================================

df_censo[['estrato','sexo','edad','peso']].head()

,estrato,sexo,edad,peso
0,caba,femenino,16,0.000246
1,caba,masculino,16,0.000247
2,caba,femenino,17,0.000257
3,caba,masculino,17,0.000257
4,caba,femenino,18,0.000286


#IMAGEN DEL CANDIDATO A LO LARGO DEL TIEMPO

In [304]:

df.set_index('fecha', inplace=True) # establecemos la columna 'fecha' como índice del DataFrame
df.sort_index(inplace=True) # ordenamos el DataFrame por el índice de fecha
#Necesitamos ordenarlas desde la más vieja a la más nueva Ahora que la fecha es el “título” de cada fila

#La idea ahora es hacer un rolling window que pueda ir uno viendo los valores de imagen del candidato a lo largo del tiempo e ir modificandola si queremos verla dentro de una semana o en 3 dias o de un dia para el otro luego de algun evento importante.


In [305]:
imagen_rolling = df["imagen_del_candidato"].rolling("7D").apply(np.mean)
print(imagen_rolling)
#usamos un apply con una función matemática (np.mean), la media, aplicado para un rwindow de 7 días
#esto nos da un promedio móvil de la imagen del candidato en una ventana de 7 días
#cada fila del resultado es el promedio de la imagen del candidato en los 7 días anteriores (incluyendo el día actual)
#esto es útil para suavizar las fluctuaciones diarias y ver tendencias a más largo plazo 


#no confundir con
# imagen_rolling = df["imagen_del_candidato"].rolling(window=7, min_periods=1).mean()
# que haría un promedio móvil simple de 7 filas, no de 7 días

fecha
2025-01-15    69.000000
2025-01-15    59.500000
2025-01-15    57.333333
2025-01-15    57.250000
2025-01-15    55.800000
                ...    
2025-03-01    76.775701
2025-03-01    76.841860
2025-03-01    76.740741
2025-03-01    76.576037
2025-03-01    76.614679
Name: imagen_del_candidato, Length: 872, dtype: float64


In [306]:
#como el dt tiene varias encuestas x día el rolling se aplica fila por fila, no por día
# cada fila es = al resultado del rolling “centrado” en esa encuesta
# entonces mejor agrupar por día ANTES del rolling para poder ver evolucion diaria y tambien en rolling window

imagen_diaria = df["imagen_del_candidato"].resample("D").mean() #usamos el sirve para crear una serie “resumida” x día.
imagen_rolling = imagen_diaria.rolling("7D").mean()

print(imagen_rolling)
#ahora si tenemos un dato por día, el promedio de todas las encuestas de ese día
#sorry si el resumen es repetitivo pero me sirve a mi para aclararme las ideas y pa cuando cuando relea el codigo 

fecha
2025-01-15    68.811111
2025-01-16    68.811111
2025-01-17    68.811111
2025-01-18    68.811111
2025-01-19    68.811111
2025-01-20    68.811111
2025-01-21    68.811111
2025-01-22          NaN
2025-01-23          NaN
2025-01-24          NaN
2025-01-25          NaN
2025-01-26          NaN
2025-01-27          NaN
2025-01-28          NaN
2025-01-29          NaN
2025-01-30          NaN
2025-01-31          NaN
2025-02-01    79.816794
2025-02-02    79.816794
2025-02-03    79.816794
2025-02-04    79.816794
2025-02-05    79.816794
2025-02-06    79.816794
2025-02-07    79.816794
2025-02-08          NaN
2025-02-09          NaN
2025-02-10          NaN
2025-02-11          NaN
2025-02-12          NaN
2025-02-13          NaN
2025-02-14          NaN
2025-02-15    66.220588
2025-02-16    66.220588
2025-02-17    66.220588
2025-02-18    66.220588
2025-02-19    66.220588
2025-02-20    66.220588
2025-02-21    66.220588
2025-02-22          NaN
2025-02-23          NaN
2025-02-24          NaN
2025-02-25

In [307]:
# ahora podemos ver una fila por dia porque el rolling se aplica a un dato por dia que es la media diaria
# sirve en casos donde hay mas de una encuesta por dia



#TEMA TRACKING INTENCION DE VOTO

In [308]:
df["voto_num"] = df["voto"].map({
    "En blanco": 0,
    "Candidato A": 1,
    "Candidato B": 2,
    "Candidato C": 3,
    "No vota": 4,
    "NS/NC": 5 #preguntar como hacer esto con los casos con o sin tilde y blabla o ns/nc??? sino queda como float
}).fillna(0)  # evitamos los NaN - justamente consultar esto x lo anterior
#pasarlo a dummies es una opcion tambien para los votos, pero no se si conviene en este caso??
print(df["voto_num"].dtype) #confirmamos que la nueva columna es numerica
df["voto_num"].head()
df["voto_num"].tail()

float64


fecha
2025-03-01    4.0
2025-03-01    3.0
2025-03-01    1.0
2025-03-01    4.0
2025-03-01    3.0
Name: voto_num, dtype: float64

In [309]:

# conteo diario de casos x candidato
conteo = df.groupby(['fecha', 'voto']).size().unstack(fill_value=0)

# total casos x día
totales = conteo.sum(axis=1)

# porcentaje diario (fórmula V_C,t / N_t * 100)
porcentaje_diario = conteo.div(totales, axis=0) * 100 # porcentaje diario de votos por candidato (como poner en % y el signo y no tantos numeros decimales?)

print(porcentaje_diario.head())
# rolling window de 7 días para suavizar
porcentaje_suavizado = porcentaje_diario.rolling("7D").mean()

voto        Candidato A  Candidato B  Candidato C  En blanco    No vota
fecha                                                                  
2025-01-15    20.000000    25.714286    28.571429  21.714286   4.000000
2025-02-01    20.233463    11.673152    14.785992  34.241245  19.066148
2025-02-15    17.525773     4.639175    30.412371  15.463918  31.958763
2025-03-01    24.074074    18.981481    20.370370  22.222222  14.351852


In [310]:
#df = df.set_index("fecha").sort_index() #indexamos por fecha y ordenamos para que pueda funcionar el rolling
#df["voto_num"].rolling("7D") # pedimor x 7D al data frame que se llama voto_num para poder aplicar el rolling
# promedio x día
#daily = df["voto_num"].resample("D").mean()

# rolling 7 días (lo podemos ir cambiando el "7D")
#tracking = daily.rolling("7D").mean()

#print(tracking) # printeamos resultado tracking
#tracking = tracking.dropna()
#print (tracking)